In [1]:
# Imports
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import tool
from pydantic import BaseModel, EmailStr, Field
os.environ["OPENAI_API_KEY"] = "sk-proj-ITGhy3LPfyvYrxbBci8pT3BlbkFJP5k2rqmOZhAdrOncxu9X"
from typing import Optional, List
import json
import csv

In [2]:
serper_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool("https://www.linkedin.com/company/sutter-health/people/")
website_search_rag_tool = WebsiteSearchTool()

In [3]:
# Contact search Agent
search_agent = Agent(
    role = 'Contact Searcher', 
    goal = 'Find the contact details of the marketing team or relevant people from a specified client.',
    backstory = 
    """
    You are a dedicated member of the marketing team at Healee, a digital healthcare company. 
    Your expertise lies in leveraging online resources to quickly gather essential information. 
    You have identified {client} as a potential client for the Healee platform. 
    Your task is to find contact information such as email addresses and LinkedIn profiles 
    of key individuals within {client}'s organization who can be approached to set up a meeting. 
    This includes members of their marketing team or other relevant stakeholders. 
    """, 
    tools = [SerperDevTool()], 
    allow_delegation = False
)

# Web Scraper agent
scraper_agent = Agent(
    role='Web Scraper',
    goal='Scrape relevant information from specified websites',
    backstory="""
    You are a skilled web scraper working for the marketing team at Healee, a digital healthcare company. 
    Your task is to extract crucial information from websites to support various marketing and research activities. 
    This includes gathering contact details, company profiles, and other relevant data from the given URLs.
    """,
    tools=[scrape_tool],
    allow_delegation=False
)

# Final Consolidator Agent
consolidator = Agent(
    role='Result Consolidator',
    goal='Combine all the contact information gathered by the scraper agents into a final, organized list of relevant contacts.',
    backstory=(
        "You are a highly skilled data synthesizer, adept at merging information from multiple sources. "
        "Your expertise lies in compiling comprehensive, clear, and concise lists of relevant contact information. "
        "Your meticulous approach ensures no detail is overlooked, providing a valuable resource for your team."
    ),
    verbose=True,
    allow_delegation=False
)


In [4]:
class PeoplePageURL(BaseModel):
    url: List[str] = Field(..., title="URL of the LinkedIn People page for the specified client.")


In [5]:
# Tasks
# Contact Search Task 
contact_search_task = Task(
    description=(
        "Use online resources to identify potential websites, LinkedIn profiles, "
        "and other online platforms where contact information of key individuals "
        "within {client}'s organization can be found. Focus on finding members "
        "of their marketing team or other relevant stakeholders. Your final list "
        "should include URLs or online resources where contact information might "
        "be available."
    ),
    expected_output=(
        "A list of at least 3 URLs or online resources where contact information "
        "of key individuals from {client} might be available. Ensure the sources "
        "are relevant and up-to-date."
    ),
    agent=search_agent
)

# Web scraping Task
web_scraping_task = Task(
    description=(
        "Scrape the provided URLs or online resources to extract contact details "
        "such as email addresses, phone numbers, and LinkedIn profiles of key "
        "individuals within {client}'s organization. Ensure the information is "
        "accurate and relevant to the marketing team or other relevant stakeholders."
    ),
    expected_output=(
        "A detailed list of contact information extracted from the provided URLs. "
        "Include names, email addresses, phone numbers, LinkedIn profiles, and "
        "any other relevant contact details."
    ),
    agent=scraper_agent
)

# Linkedin people page task
linkedin_people_page_search_task = Task(
    description=(
        "Use online resources, including LinkedIn, to identify the 'People' page URL "
        "for {client}'s organization on LinkedIn. For example, if the LinkedIn page of a company "
        "is 'https://www.linkedin.com/company/emplomind/', then the 'People' page URL is "
        "'https://www.linkedin.com/company/emplomind/people/'. Focus on finding the page where "
        "LinkedIn profiles of the company's employees are listed. Your final output should include "
        "the URL of the LinkedIn 'People' page."
    ),
    expected_output=(
        "The URL of the LinkedIn 'People' page for {client}'s organization."
    ),
    agent=search_agent, 
    output_model=PeoplePageURL
)



# Linkedin people page scrape task
linkedin_profile_scraping_task = Task(
    description=(
        "Scrape the provided LinkedIn 'People' page URL to extract LinkedIn profiles of key "
        "individuals within {client}'s organization. For example, if the LinkedIn page of a company "
        "is 'https://www.linkedin.com/company/emplomind/', the scraper should focus on the "
        "'https://www.linkedin.com/company/emplomind/people/' page. Focus on finding profiles of "
        "members of their marketing team or other relevant stakeholders. Ensure the information is accurate "
        "and relevant to the marketing team or other stakeholders."
    ),
    expected_output=(
        "A detailed list of LinkedIn profiles extracted from the provided LinkedIn 'People' page URL. "
        "Include names, LinkedIn profile URLs, and roles within the company."
    ),
    agent=scraper_agent
)


In [6]:
# Pydantic for the final consolidation of results
class ContactInfo(BaseModel):
    name: str
    role_in_company: str
    email_address: Optional[EmailStr] = Field(None, alias='email')
    linkedin: Optional[str] = None
    phone_no: Optional[str] = None

# Consolidation task function
# def consolidation_task_function(results: List[ContactInfo]) -> str: 
#     # Convert the list of ContactInfo objects to JSON
#     json_output = json.dumps([results.dict() for result in results], indent=2)

#     # Convert the list of ContactInfo objects to CSV
#     csv_output = "name,role_in_company,email_address,linkedin,phone_no\n"
#     for result in results:
#         csv_output += f"{result.name},{result.role_in_company},{result.email_address or ''},{result.linkedin or ''},{result.phone_no or ''}\n"
    
#     with open("contacts.csv", "w") as csv_file:
#         csv_file.write(csv_output)

#     return "Consolidation complete. Outputs saved to contacts.json and contacts.csv."

def consolidation_task_function(results: List[dict]) -> str:
    contacts = [ContactInfo(**result) for result in results]
    
    # Convert to JSON
    # json_output = [contact.dict() for contact in contacts]
    # json_file_path = "./contacts.json"
    # with open(json_file_path, "w") as json_file:
    #     json.dump(json_output, json_file, indent=2)
    
    # Convert to CSV
    csv_file_path = "./contacts.csv"
    with open(csv_file_path, "w", newline='') as csv_file:
        fieldnames = ['name', 'role_in_company', 'email_address', 'linkedin', 'phone_no']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for contact in contacts:
            writer.writerow(contact.dict())
    
    return f"Consolidation complete. Outputs saved to contacts.json and contacts.csv."

In [7]:
# Final Consolidation task
consolidation_task = Task(
    description=(
        "Combine all the contact information gathered by the scraper agents into a final, organized list of relevant contacts. "
        "The output should be formatted as CSV files with the fields: Name, Role in company, Email address, LinkedIn, Phone no. "
        "Leave any of Email address, LinkedIn, Phone no. empty if the information is not found."
    ),
    expected_output=(
        "A CSV file containing the consolidated list of contact information. The fields should include: Name, Role in company, Email address, LinkedIn, Phone no."
    ),
    agent = consolidator, 
    function = consolidation_task_function, 
    output_file='contacts.csv'
)

In [8]:
# Create the crew
crew = Crew(
    agents=[search_agent, scraper_agent, consolidator], 
    tasks=[linkedin_people_page_search_task, linkedin_profile_scraping_task, consolidation_task],
    process=Process.sequential
)

result = crew.kickoff(inputs={'client': 'Sutter Health'})
print(result)

 


Search results: Title: Sutter Health | LinkedIn
Link: https://www.linkedin.com/company/sutter-health
Snippet: Sutter Health | 157151 followers on LinkedIn. An integrated network delivering accessible, high-quality, and life-saving healthcare when ...
---
Title: Sutter Health - Bay Area - LinkedIn
Link: https://www.linkedin.com/company/sutterhealthbayarea
Snippet: Sutter Health - Bay Area, Modesto, CA ... We're #hiring a new Nurse Practitioner - Internal Medicine - Stockton, CA in Stockton, California. Apply today or share ...
---
Title: Sutter Medical Group - LinkedIn
Link: https://www.linkedin.com/company/sutter-medical-group
Snippet: Sutter Medical Group | 540 followers on LinkedIn. Sutter Medical Group is a successful, 800+ member multi-specialty group offering physicians the ...
---
Title: Sutter Health Plus | LinkedIn
Link: https://www.linkedin.com/company/sutter-health-plus
Snippet: We provide coverage for hospitalization, outpatient services, prescription drug coverage and n

In [12]:
! pip install webdriver_manager

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
